## Introduction

While there are a number of coffee shops throughout Toronto, it is important to focus precisely on a particular market to be successful. For example a high end, high throughput shop with early opening and closing hours might work well in a central business district, while something that offers more seating and greater meal selection and is open late might appeal to a residential area. Proximity to amenities that are open late, such as movie of dance theaters, may draw enough sophisticated consumers to justify a loss of early morning traffic. For this exercise, I would like to locate an optimal area in which to open a small coffee shop. 

## Data

I will be using data from FourSquare and latitude and longitude data on Toronto neighbourhoods from the University of Toronto digital archives. The FourSquare data will include venues as in the lab 3-3-2, "Segmenting and Clustering Neighbourhoods in New York City", and the geojson data from the U of T includes a summary of the neighbourhoods around Toronto. I will use a combination of these data to locate an area within the city that is currently underserved.

Load all required libraries for notebook:

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle JSON files
import matplotlib.cm as cm
import folium # plotting library
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image
from IPython.core.display import HTML 
from pandas import json_normalize
from sklearn.cluster import KMeans

Load Foursquare client_ID and client_secret.

In [2]:
CLIENT_ID = 'N14OLCHFJEKBXDWPA55MSTTPSVNO25ZU2MGCSU05T0TVCO5L' # your Foursquare ID
CLIENT_SECRET = 'EW4R0WJOIM51QXGZ5WJL5E3WKLGR0MBJPIIN1NSGWJ0J1RMZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 60

For the first exercise, we will see what coffee shops and other entertainment options are available around the CN Tower in Toronto. 

In [3]:
latitude = 43.6426
longitude = -79.3871

In [4]:
# Function that extracts the category of the venue.
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

We would like to search for existing coffee shops, banks, offices, and entertainment venues. We don't want our coffee shop too close to an existing cluster of coffee shops, however, we don't want it too far away from any amenities (e.g. office buildings, entertainment locations, banks). We will define our search list. 

In [5]:
search_list = ['coffee','banks','offices','entertainment']

We will now search an area within 10km of the CN tower for the types of businesses we specified in our search list. 

In [6]:
# Initialize and zero out all variables and dataframes. 
df_venuefull = []
dfs = {}
i = "0"
j = 0

for i in search_list:

    # Define what to search for (search_query) and generate result
    search_query = i
    radius = 10000
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()

    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    dataframe.head()

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']

    dataframe_filtered = dataframe.loc[:, filtered_columns]
    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    dfs.update({str(i):dataframe_filtered})
    
    df_venuefull.append(dataframe_filtered)  

We will now generate a map, centered around the CN tower (red dot), showing the coffee shops in blue, banks in green, offices in orange and entertainment venues in purple.

In [7]:
 df_venuefull = pd.concat(df_venuefull, sort=True)

In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add a red circle marker to represent the CN Tower
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='CN Tower',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the coffee shops as blue circle markers
for lat, lng, label in zip(dfs['coffee'].lat, dfs['coffee'].lng, dfs['coffee'].categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the banks shops as green circle markers
for lat, lng, label in zip(dfs['banks'].lat, dfs['banks'].lng, dfs['banks'].categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the offices as orange circle markers
for lat, lng, label in zip(dfs['offices'].lat, dfs['offices'].lng, dfs['offices'].categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the entertainment as purple circle markers
for lat, lng, label in zip(dfs['entertainment'].lat, dfs['entertainment'].lng, dfs['entertainment'].categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.6
    ).add_to(venues_map)

In [9]:
venues_map

Information on toronto neighbourhoods was downloaded from the U of T data science portal. It is included in the file 'Toronto_Neighbourhoods.geojson'. It will now be loaded into toronto_data in json format. 

In [10]:
with open('Toronto_Neighbourhoods.geojson') as json_data:
    toronto_data = json.load(json_data)

Have a look at the loaded data to see what is contained in the file. 

In [11]:
toronto_data

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'_id': 2661,
    'AREA_ID': 25886861,
    'AREA_ATTR_ID': 25926662,
    'PARENT_AREA_ID': 49885,
    'AREA_SHORT_CODE': 94,
    'AREA_LONG_CODE': 94,
    'AREA_NAME': 'Wychwood (94)',
    'AREA_DESC': 'Wychwood (94)',
    'X': None,
    'Y': None,
    'LONGITUDE': -79.425514947,
    'LATITUDE': 43.6769192679,
    'OBJECTID': 16491505,
    'Shape__Area': 3217959.609375,
    'Shape__Length': 7515.779658331329},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-79.43591570873059, 43.68015339477487],
      [-79.43491506339724, 43.68036886994886],
      [-79.43394727223847, 43.680578504490335],
      [-79.43388162422195, 43.68058996121469],
      [-79.43281544978878, 43.680808044458765],
      [-79.4326971769691, 43.68079658822322],
      [-79.43253064659865, 43.68082785962871],
      [-79.43245944449654, 43.6808588271

We will create a blank dataframe to accomodate the name, ID, area and lat/long for each neighbourhood in Toronto.

In [12]:
neighbourhoods_data = toronto_data['features']
# define the dataframe columns
column_names = ['AreaName', 'AreaID', 'ShapeArea', 'Longitude', 'Latitude'] 
# instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)

Populate the neighbourhoods dataframe from the geojson data loaded earlier.

In [13]:
for data in neighbourhoods_data:
    area_name = data['properties']['AREA_NAME'] 
    area_ID = data['properties']['AREA_ID']
    shape_area = data['properties']['Shape__Area']
    neighbourhood_lat = data['properties']['LATITUDE']
    neighbourhood_lon = data['properties']['LONGITUDE']
    
    neighbourhoods = neighbourhoods.append({'AreaName': area_name,
                                          'AreaID': area_ID,
                                          'ShapeArea': shape_area,
                                          'Latitude': neighbourhood_lat,
                                          'Longitude': neighbourhood_lon}, ignore_index=True)

In [1]:
# Function that takes a name, latitude, longitude and search radius, and returns a dataframe containing a 
# list of Foursquare venues for those locations. 

def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We will perform a FourSquare venue search on each one of the neighbourhoods and populate the toronto_venues dataframe with these results.

In [15]:
toronto_venues = getNearbyVenues(names=neighbourhoods['AreaName'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )

Wychwood (94)
Yonge-Eglinton (100)
Yonge-St.Clair (97)
York University Heights (27)
Yorkdale-Glen Park (31)
Lambton Baby Point (114)
Lansing-Westgate (38)
Lawrence Park North (105)
Lawrence Park South (103)
Leaside-Bennington (56)
Little Portugal (84)
Long Branch (19)
Malvern (132)
Maple Leaf (29)
Markland Wood (12)
Milliken (130)
Mimico (includes Humber Bay Shores) (17)
Morningside (135)
Moss Park (73)
Mount Dennis (115)
Mount Olive-Silverstone-Jamestown (2)
Mount Pleasant East (99)
Mount Pleasant West (104)
New Toronto (18)
Newtonbrook East (50)
Newtonbrook West (36)
Niagara (82)
North Riverdale (68)
North St.James Town (74)
O'Connor-Parkview (54)
Oakridge (121)
Oakwood Village (107)
Old East York (58)
Palmerston-Little Italy (80)
Parkwoods-Donalda (45)
Pelmo Park-Humberlea (23)
Playter Estates-Danforth (67)
Pleasant View (46)
Princess-Rosethorn (10)
Regent Park (72)
Rexdale-Kipling (4)
Rockcliffe-Smythe (111)
Roncesvalles (86)
Rosedale-Moore Park (98)
Rouge (131)
Runnymede-Bloor Wes

Results in the dataframe will then be one-hot endoded against all of the venues. We will then group all of the venues together to 

In [16]:
# one hot encoding
toronto_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_venues_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_venues_onehot.columns[-1]] + list(toronto_venues_onehot.columns[:-1])
toronto_venues_onehot = toronto_venues_onehot[fixed_columns]

toronto_venues_onehot.head()

toronto_grouped = toronto_venues_onehot.groupby('Neighbourhood').mean().reset_index()


Have a look at the grouped data.

In [17]:
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt North (129),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
1,Agincourt South-Malvern West (128),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
2,Alderwood (20),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016667,...,0.0,0.0,0.0,0.0,0.016667,0.000000,0.0,0.000000,0.0,0.0
3,Annex (95),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.016667,0.0,0.0
4,Banbury-Don Mills (42),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.016667,0.0,0.000000,0.0,0.0


In [18]:
# We only want to see the top ten venues for each cluster. 
num_top_venues = 10

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.describe()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,140,140,140,140,140,140,140,140,140,140,140
unique,140,28,31,39,53,58,55,63,66,71,71
top,Greenwood-Coxwell (65),Coffee Shop,Coffee Shop,Coffee Shop,Sandwich Place,Grocery Store,Restaurant,Sandwich Place,Bank,Liquor Store,Gastropub
freq,1,50,31,21,11,11,10,9,8,8,7


## Clustering

We will use k-means clustering to group all the venues into categories for further analysis.

In [20]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
df_kmeansc = pd.DataFrame(kmeans.cluster_centers_)

We'll then create a dictionary that will compare the kmeans clusered data to the Toronto data that includes neighbourhood names and locations. 

In [21]:
ren_col = dict(zip(list(df_kmeansc.columns), list(toronto_grouped_clustering.columns))) 

In [23]:
df_kmeans = df_kmeansc.rename(columns = ren_col)
df_kmeans.head()

,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Art Gallery,...,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,-1.355253e-20,0.000000e+00,0.000000e+00,5.421011e-20,0.004762,2.380952e-03,0.004762,-2.710505e-20,-5.421011e-20,2.168404e-19,...,0.000000e+00,2.380952e-03,4.761905e-03,0.000000,4.336809e-19,0.000000e+00,0.000000e+00,0.004762,1.084202e-19,0.0000
1,-1.355253e-20,0.000000e+00,2.168404e-19,-1.084202e-19,0.005833,-2.710505e-20,0.001667,-2.710505e-20,-5.421011e-20,-4.336809e-19,...,1.666667e-03,-5.421011e-20,6.666667e-03,0.000000,2.500000e-03,-3.252607e-19,-3.252607e-19,0.015000,-3.252607e-19,0.0000
2,-4.065758e-20,2.168404e-19,2.168404e-19,-5.421011e-20,0.008936,-8.131516e-20,0.000000,-8.131516e-20,-5.421011e-20,1.644737e-03,...,1.041667e-03,-1.626303e-19,-2.168404e-19,0.000000,3.125000e-03,-2.168404e-19,2.083333e-03,0.000000,-2.168404e-19,0.0000
3,0.000000e+00,0.000000e+00,3.252607e-19,-5.421011e-20,0.001675,0.000000e+00,0.000000,0.000000e+00,-5.421011e-20,-4.336809e-19,...,-6.505213e-19,0.000000e+00,2.168404e-19,0.000952,4.386167e-03,1.190476e-03,-2.168404e-19,0.000000,-2.168404e-19,0.0000
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,6.250000e-02,0.5625


In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = neighbourhoods.merge(neighborhoods_venues_sorted, left_on='AreaName', right_on = 'Neighbourhood')

toronto_merged.head(10) # check the last columns!

,AreaName,AreaID,ShapeArea,Longitude,Latitude,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wychwood (94),25886861,3.217960e+06,-79.425515,43.676919,1,Wychwood (94),Café,Italian Restaurant,Coffee Shop,Park,Indian Restaurant,Ice Cream Shop,Mexican Restaurant,Bar,South American Restaurant,Chinese Restaurant
1,Yonge-Eglinton (100),25886820,3.160334e+06,-79.403590,43.704689,1,Yonge-Eglinton (100),Italian Restaurant,Coffee Shop,Park,Café,Gym,Gastropub,Bookstore,Restaurant,Bakery,Deli / Bodega
2,Yonge-St.Clair (97),25886834,2.222464e+06,-79.397871,43.687859,1,Yonge-St.Clair (97),Italian Restaurant,Park,Sushi Restaurant,Grocery Store,Café,Mexican Restaurant,Spa,Bagel Shop,Liquor Store,Coffee Shop
3,York University Heights (27),25886593,2.541821e+07,-79.488883,43.765736,3,York University Heights (27),Coffee Shop,Gas Station,Furniture / Home Store,Sandwich Place,Grocery Store,Pizza Place,Restaurant,Bank,Pharmacy,Middle Eastern Restaurant
4,Yorkdale-Glen Park (31),25886688,1.156669e+07,-79.457108,43.714672,9,Yorkdale-Glen Park (31),Clothing Store,Furniture / Home Store,Coffee Shop,Cosmetics Shop,Toy / Game Store,Fried Chicken Joint,Food Court,Men's Store,Fast Food Restaurant,Greek Restaurant
5,Lambton Baby Point (114),25886404,3.406857e+06,-79.496045,43.657420,1,Lambton Baby Point (114),Bakery,Italian Restaurant,Coffee Shop,Pizza Place,Café,Burger Joint,Dessert Shop,French Restaurant,Sushi Restaurant,Gastropub
6,Lansing-Westgate (38),25886627,1.025436e+07,-79.424748,43.754271,7,Lansing-Westgate (38),Coffee Shop,Grocery Store,Café,Restaurant,Pharmacy,Japanese Restaurant,Burger Joint,Bank,Gym,Dog Run
7,Lawrence Park North (105),25886808,4.388577e+06,-79.403978,43.730060,5,Lawrence Park North (105),Coffee Shop,Park,Café,Sushi Restaurant,Sporting Goods Shop,Bakery,Restaurant,Italian Restaurant,Japanese Restaurant,Pub
8,Lawrence Park South (103),25886812,6.211341e+06,-79.406039,43.717212,1,Lawrence Park South (103),Italian Restaurant,Coffee Shop,Café,Bakery,Park,Bookstore,Yoga Studio,Burger Joint,Gym,Movie Theater
9,Leaside-Bennington (56),25886328,9.161864e+06,-79.366072,43.703797,7,Leaside-Bennington (56),Coffee Shop,Bakery,Indian Restaurant,Park,Sushi Restaurant,Café,Restaurant,Thai Restaurant,Gym,Grocery Store


In [27]:
list(toronto_merged.columns) 
toronto_merged_list = [('1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue','5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue')]

We'll now create a map of the coded venues. 

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['AreaName'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
toronto_merged.head()

,AreaName,AreaID,ShapeArea,Longitude,Latitude,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wychwood (94),25886861,3.217960e+06,-79.425515,43.676919,1,Wychwood (94),Café,Italian Restaurant,Coffee Shop,Park,Indian Restaurant,Ice Cream Shop,Mexican Restaurant,Bar,South American Restaurant,Chinese Restaurant
1,Yonge-Eglinton (100),25886820,3.160334e+06,-79.403590,43.704689,1,Yonge-Eglinton (100),Italian Restaurant,Coffee Shop,Park,Café,Gym,Gastropub,Bookstore,Restaurant,Bakery,Deli / Bodega
2,Yonge-St.Clair (97),25886834,2.222464e+06,-79.397871,43.687859,1,Yonge-St.Clair (97),Italian Restaurant,Park,Sushi Restaurant,Grocery Store,Café,Mexican Restaurant,Spa,Bagel Shop,Liquor Store,Coffee Shop
3,York University Heights (27),25886593,2.541821e+07,-79.488883,43.765736,3,York University Heights (27),Coffee Shop,Gas Station,Furniture / Home Store,Sandwich Place,Grocery Store,Pizza Place,Restaurant,Bank,Pharmacy,Middle Eastern Restaurant
4,Yorkdale-Glen Park (31),25886688,1.156669e+07,-79.457108,43.714672,9,Yorkdale-Glen Park (31),Clothing Store,Furniture / Home Store,Coffee Shop,Cosmetics Shop,Toy / Game Store,Fried Chicken Joint,Food Court,Men's Store,Fast Food Restaurant,Greek Restaurant


Lets now look at the properly grouped data with most common venues listed. Utilizing this table and the map allows us to make the determination that placing a coffee shop in cluster 4 is prospective as that looks to be a currently underserved market. 

In [30]:
toronto_merged.groupby(['Cluster Labels']).agg(lambda x:x.value_counts().index[0])

,AreaName,AreaID,ShapeArea,Longitude,Latitude,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,
0,Regent Park (72),25886941,3.598782e+06,-79.466302,43.657565,Regent Park (72),Café,Coffee Shop,Bar,Italian Restaurant,Gastropub,Restaurant,Dog Run,Performing Arts Venue,Pub,Gastropub
1,Weston-Pellam Park (91),25886818,1.518135e+07,-79.384924,43.704435,Weston-Pellam Park (91),Italian Restaurant,Coffee Shop,Coffee Shop,Café,Bar,Pizza Place,Indian Restaurant,Burger Joint,Liquor Store,Gastropub
2,Pleasant View (46),25886494,1.511736e+07,-79.354804,43.788658,Pleasant View (46),Coffee Shop,Clothing Store,Chinese Restaurant,Bakery,Chinese Restaurant,Caribbean Restaurant,Sandwich Place,Sandwich Place,Juice Bar,Japanese Restaurant
3,Mount Dennis (115),25886548,6.643702e+06,-79.522416,43.683645,Mount Dennis (115),Coffee Shop,Park,Park,Pizza Place,Sandwich Place,Fast Food Restaurant,Bank,Bank,Grocery Store,Pharmacy
4,Rouge (131),25886407,7.214402e+07,-79.186343,43.821201,Rouge (131),Zoo Exhibit,Zoo,Gas Station,Trail,Dessert Shop,Other Great Outdoors,Grocery Store,Garden Center,Pizza Place,National Park
5,Greenwood-Coxwell (65),25886844,3.416312e+06,-79.467872,43.679700,Greenwood-Coxwell (65),Café,Coffee Shop,Park,Coffee Shop,Coffee Shop,Bar,Cocktail Bar,Bar,Taco Place,Pub
6,Kingsview Village-The Westway (6),25886697,6.402351e+06,-79.498091,43.737988,Kingsview Village-The Westway (6),Coffee Shop,Coffee Shop,Fast Food Restaurant,Sandwich Place,Indian Restaurant,Restaurant,Bank,Bank,Chinese Restaurant,Bank
7,St.Andrew-Windfields (40),25886718,9.502180e+06,-79.349984,43.670886,St.Andrew-Windfields (40),Coffee Shop,Bank,Grocery Store,Beer Store,Pharmacy,Café,Burger Joint,Fast Food Restaurant,Gym,Beer Store
8,Willowdale West (37),25886598,9.015928e+06,-79.401484,43.770602,Willowdale West (37),Korean Restaurant,Bubble Tea Shop,Coffee Shop,Café,Bubble Tea Shop,Sushi Restaurant,Sushi Restaurant,Fried Chicken Joint,Theater,Thai Restaurant
